In [ ]:
%load_ext autoreload
%autoreload 2
import os

os.environ['SM_FRAMEWORK'] = 'tf.keras'

In [ ]:

from pathlib import Path
import tensorflow as tf
import cv2
from PIL import Image
import numpy as np
from matplotlib import cm
from PIL.Image import BICUBIC

import math
import scipy
import matplotlib.pyplot as plt
from watch_recognition.utilities import predictions_to_polygon
from watch_recognition.targets_encoding import select_hand_points_with_line_fits
from skimage.color import label2rgb
from skimage.measure import label, regionprops, find_contours
import matplotlib.patches as mpatches
from matplotlib import pyplot as plt
from skimage.feature import corner_harris, corner_subpix, corner_peaks
from functools import lru_cache
from PIL import ImageOps
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.morphology import erosion, skeletonize, thin
from skimage.measure import regionprops
from skimage import measure
from skimage.transform import hough_line, hough_line_peaks
from matplotlib import cm
from skimage.filters import gaussian
from watch_recognition.utilities import Point, Line
from IPython.display import display
from PIL import ImageOps
from skimage.feature import canny
from skimage.transform import probabilistic_hough_line
from sklearn.cluster import KMeans
import numpy as np
from watch_recognition.utilities import mean_line, minmax_line
from typing import List
from collections import defaultdict
from skimage.morphology import skeletonize
from watch_recognition.targets_encoding import convert_mask_outputs_to_keypoints
from skimage.transform import rescale
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft2, fftshift
from skimage.draw import line as draw_line
from skimage import img_as_float
from skimage.color import rgb2gray
from skimage.data import astronaut
from watch_recognition.data_preprocessing import load_image
from skimage.measure import label


%matplotlib inline


In [ ]:
keypoint_model = '../cloud_train/local-training-output/heatmap-regression_098DB017/models/keypoint/efficientnetb0-unet-96-hands/run_1636479122.611962/'
# keypoint_model = '../models/run_1635107772.736066'
# keypoint_model = '../models/keypoint/efficientnetb0-unet-sigmoid-128/run_1635363092.476276'
sm_model_path = "../cloud_train/local-training-output/segmentation/segmentation_48D99B6B/models/segmentation/efficientnetb0-unet-(96, 96)-391-aug/export/"
sm_model = tf.keras.models.load_model(sm_model_path, compile=False)
kp_model = tf.keras.models.load_model(keypoint_model, compile=False)
image_size = tuple(sm_model.inputs[0].shape[1:3])

In [ ]:


def plot_kp_on_image(original_image, point: Point):
    # Plot the detection results on the input image
    original_image_np = original_image.astype(np.uint8)

    x, y = point.as_coordinates_tuple
    x = int(x)
    y = int(y)

    # Draw the bounding box and label on the image
    cv2.drawMarker(original_image_np, (x, y), (255, 0, 0), cv2.MARKER_CROSS,
                   thickness=1, markerSize=3)

    # Return the final image
    original_uint8 = original_image_np.astype(np.uint8)
    return original_uint8


def plot_line_on_image(original_image, line: Line):
    original_image_np = original_image.astype(np.uint8)

    start = line.start.as_coordinates_tuple
    end = line.end.as_coordinates_tuple
    start = tuple(map(int, start))
    end = tuple(map(int, end))
    cv2.line(original_image_np, start, end, (0, 255, 0))
    # Return the final image
    original_uint8 = original_image_np.astype(np.uint8)
    return original_uint8

In [ ]:
file = sorted(Path("./frames/").glob("*_0.jpg"))[0]
with Image.open(file) as crop:
    display(crop)

In [ ]:
crop_rotated_resized_np = ImageOps.pad(
    crop,
    image_size, BICUBIC
)
frame = np.array(crop_rotated_resized_np).copy()
# keypoints
image_np = np.expand_dims(crop_rotated_resized_np, 0)
predicted = sm_model.predict(
    image_np
)[0]


predicted_kp = kp_model.predict(image_np)[0]
center = convert_mask_outputs_to_keypoints(predicted_kp,
                                            decode_hands_from_lines=False)[0]

In [ ]:
plt.imshow(predicted)

In [ ]:

image = predicted.copy()

image = rescale(image, 5, anti_aliasing=True, )
image = image[:,:,0].squeeze()
image = (image > 0.1).astype(int)
print(image.shape)

image_f = np.abs(fftshift(fft2(image)))

fig, axes = plt.subplots(1, 2, figsize=(8, 8))
ax = axes.ravel()
ax[0].set_title("Original image")
ax[0].imshow(image, cmap='gray')
ax[1].set_title("Original FFT (frequency)")
ax[1].imshow(np.log(image_f), cmap='magma')

plt.show()

In [ ]:
from watch_recognition.utilities import Line, mean_line
def kmeans_cluster_lines(lines: List[Line], n_clusters=3):
    xs = []
    n_features = 0
    for line in lines:

        x = np.hstack((line.unit_vector, line.center.as_array))
        x = np.append(x,line.slope)
        n_features = len(x)
        xs.append(x)

    X = np.array(xs)
    X = X.reshape(-1, n_features)
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
    clusters = defaultdict(list)
    for i, label in enumerate(kmeans.labels_):
        l = lines[i]
        clusters[label].append(l)


    return clusters

def probabilistic_hugh_transform(image, center, debug=False):
    image = image.squeeze()
    thresholded_image = (image > 0.1).astype('bool')

    label_image = label(thresholded_image)

    image_label_overlay = label2rgb(label_image, image=thresholded_image, bg_label=0)
    # select the largest object to filter out small false positives
    region = sorted(regionprops(label_image), key=lambda r: r.area, reverse=True)[0]
    region_image = label_image == region.label
    # image = canny(image)
    line_length =int( np.sqrt(region.bbox_area) / 3)


    lines = probabilistic_hough_line(region_image, line_length=line_length, threshold=1, seed=42, line_gap=10)

    new_lines = []
    for start, end in lines:
        start_p = Point(*start)
        end_p = Point(*end)
        origin = Point(0, 0)
        if end_p.distance(origin) < start_p.distance(origin):
            end_p, start_p = start_p, end_p
        l = Line(start_p, end_p)
        new_lines.append(l)
    lines = new_lines
    if not lines:
        raise ValueError
    clustered_lines = kmeans_cluster_lines(lines)
    # clustered_lines = []
    if debug:
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        ax = axes.ravel()
        for l in lines:
            l.plot(ax=ax[0])
        ax[0].imshow(image, cmap=cm.gray)
        ax[0].set_title('Input image')
        center.plot(ax[0])
        idx_to_color = {0: 'g', 1:'r', 2:'m'}
        for i, lines in clustered_lines.items():
            l = mean_line(lines)
            l.plot(ax[1], lw=5, color=idx_to_color[i])
            for line in lines:
                line.plot(ax[1], color=idx_to_color[i])

        ax[1].imshow(image, cmap=cm.gray)
        ax[1].set_xlim((0, image.shape[1]))
        ax[1].set_ylim((image.shape[0], 0))
        ax[1].set_title('Probabilistic Hough')
        center.plot(ax[1])

        for a in ax:
            a.set_axis_off()

        plt.tight_layout()
        plt.show()
    return lines

probabilistic_hugh_transform(predicted, center, debug=True)

In [ ]:
plt.imshow(predicted)

In [ ]:

test_img = load_image('../example_data/Zrzut ekranu 2021-08-25 o 22.24.14.jpg', image_size=image_size)

In [ ]:
plt.imshow(test_img)

In [ ]:
predicted = sm_model.predict(np.expand_dims(test_img, 0))[0]

In [ ]:
plt.imshow(predicted)

In [ ]:
predicted.shape

In [ ]:


poly = predictions_to_polygon(predicted, debug=True, approximation_tolerance=1)

In [ ]:
def extract_points_from_predictions(predicted_img, center: Point, debug=False):
    predicted_img = predicted_img.squeeze()
    thresholded_image = (predicted_img > 0.1)
    label_image = label(thresholded_image)

    image_label_overlay = label2rgb(label_image, image=thresholded_image, bg_label=0)
    # select the largest object to filter out small false positives
    region = sorted(regionprops(label_image), key=lambda r: r.area, reverse=True)[0]
    region_image = label_image == region.label

    minr, minc, maxr, maxc = region.bbox
    rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                              fill=False, edgecolor='red', linewidth=2)

    coords = corner_peaks(corner_harris(region_image), min_distance=3, threshold_rel=0.1)

    # select only convex cornes (their position is the True pixels)
    coords = np.array([coord for coord in coords if np.all(region_image[coord[0], coord[1]])])

    hand_points_proposals = [Point(coord[1], coord[0]) for coord in coords]

    hands = select_hand_points_with_line_fits(center, hand_points_proposals, max_distance=10)

    if debug:
        plt.imshow(predicted_img, cmap=cm.gray)
        plt.show()
        plt.imshow(thresholded_image, cmap=cm.gray)
        plt.show()

        fig, ax = plt.subplots(figsize=(10, 6))
        center.plot(ax)
        ax.imshow(image_label_overlay)
        fig, ax = plt.subplots(figsize=(10, 6))
        center.plot(ax)
        ax.imshow(region_image, cmap=cm.gray_r)
        ax.add_patch(rect)
        for hand in hand_points_proposals:
            hand.plot(ax=ax, color='r', marker='x', size=50)

        for hand in hands:
            hand.plot(ax=ax, color='g', marker='x', size=50)
    return hands

extract_points_from_predictions(predicted, center=center, debug=True)

In [ ]:

def hugh_transform(predicted_img, center: Point, debug):
    predicted_img = predicted_img.squeeze()

    blurred_image = gaussian(predicted_img)

    thresholded_image = (predicted_img > 0.1)
    blurred_thresholded_image = (blurred_image > 0.1)

    skeletonized_image = skeletonize(predicted_img)

    center_image = np.zeros_like(predicted_img)
    int_center = tuple(map(int, map(np.round, center.as_coordinates_tuple)))
    center_image[int_center[1], int_center[0]] = 1



    tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360, endpoint=False)

    h1, theta1, d1 = hough_line(thresholded_image, theta=tested_angles, )
    h2, theta2, d2 = hough_line(blurred_thresholded_image, theta=tested_angles, )
    h3, theta3, d3 = hough_line(skeletonized_image, theta=tested_angles, )

    h = h1+h2+h3
    theta = theta1#+theta2
    d = d1#+d2


    angle_step = 0.5 * np.diff(theta).mean()
    d_step = 0.5 * np.diff(d).mean()
    bounds = [np.rad2deg(theta[0] - angle_step),
              np.rad2deg(theta[-1] + angle_step),
              d[-1] + d_step, d[0] - d_step]

    kernel = np.array([
        [1,2,1],
        [2,4,2],
        [1,2,1],
    ]   )

    h = scipy.signal.convolve2d(h, kernel, mode='same')
    h = h/16
    m = h.max()
    h = h/m
    feature_map = np.log(1 + h)
    feature_map_1 = feature_map > 0.2

    # feature_map_3 = feature_map > 0.5
    feature_map = feature_map_1



    # feature_map = np.where(feature_map> 0.5, feature_map, np.zeros_like(feature_map))

    fig, axes = plt.subplots(1, 3, figsize=(15, 6))
    ax = axes.ravel()


    label_image = label(feature_map_1)
    image_label_overlay = label2rgb(label_image, image=feature_map_1, bg_label=0)
    # select the largest object to filter out small false positives
    ax[1].imshow(image_label_overlay)
    regions = regionprops(label_image)
    regions = [region for region in regions if region.area > 50]
    for region in regions:
        y0, x0 = region.centroid
        ax[1].scatter(x0, y0)
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                              fill=False, edgecolor='red', linewidth=2)

        # print(region.minor_axis_length)


        ax[1].add_patch(rect)
        angle = theta[int(x0)]
        dist = d[int(y0)]
        (x0, y0) = dist * np.array([np.cos(angle), np.sin(angle)])
        # print(x0, y0)
        # print((np.exp(feature_map_1[int(x0), int(y0)])-1)*m)
        # print()
        slope = np.tan(angle + np.pi / 2)
        ax[2].axline((x0, y0), slope=slope)



    ax[0].imshow(thresholded_image, cmap=cm.gray_r)
    ax[0].set_title('Input image')
    ax[0].set_axis_off()
    # ax[0].scatter(*int_center)


    # ax[1].imshow(feature_map, extent=bounds, cmap=cm.gray_r, aspect=1 / 1.5)
    ax[1].set_title('Hough transform')
    ax[1].set_xlabel('Angles (degrees)')
    ax[1].set_ylabel('Distance (pixels)')
    ax[1].axis('image')

    ax[2].imshow(thresholded_image, cmap=cm.gray_r)
    ax[2].set_ylim((thresholded_image.shape[0], 0))
    ax[2].set_axis_off()
    ax[2].set_title('Detected lines')
    # ax[2].scatter(*int_center)

    peaks = hough_line_peaks(h, theta, d, num_peaks=3)
    peaks = list(zip(*peaks))
    peaks = sorted(peaks, key=lambda x: x[0], reverse=True)[:3] # select top 3 lines by peak intensity

    # for intensity, angle, dist in peaks:
    #     print(intensity)
    #     ax[1].scatter(np.rad2deg(angle), dist, marker='x')
    #     (x0, y0) = dist * np.array([np.cos(angle), np.sin(angle)])
    #     ax[2].axline((x0, y0), slope=np.tan(angle + np.pi / 2))

    plt.tight_layout()
    plt.show()
    # fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    # X, Y = np.meshgrid(theta, d)
    # print(theta)
    # feature_map = feature_map[1:-1, 1:-1]
    # bottom = np.zeros_like(feature_map)
    # width = angle_step
    # depth = d_step
    # top =feature_map
    # ax.bar3d(X.ravel(), Y.ravel(), bottom.ravel(), width, depth, top.ravel(), shade=True)

    # surf = ax.plot_surface(X, Y, feature_map, cmap=cm.coolwarm,
    #                    linewidth=0, antialiased=True,)
    # # Add a color bar which maps values to colors.
    # plt.show()




hugh_transform(predicted, center=center, debug=True)

In [ ]:
from skimage.feature import canny
from skimage.transform import probabilistic_hough_line

from watch_recognition.utilities import mean_line, minmax_line
from typing import List
from collections import defaultdict

def cluster_lines(line_proposals: List[Line]):

    line_clusters = defaultdict(list)
    for i, line in enumerate(line_proposals):
        for line_id, lines in line_clusters.items():
            mean_slope = np.mean([l.unit_vector for l in lines], axis=0)
            if mean_slope @ line.unit_vector.T > 0.99:
                line_clusters[line_id].append(line)
                break
        else:
            line_clusters[i].append(line)
    # print(f"n clusters {len(line_clusters)}")
    num_lines = 0
    for i, lines in line_clusters.items():
        num_lines += len(lines)

    assert num_lines == len(line_proposals)
    ok_lines = []
    for i, lines in line_clusters.items():
        if len(lines) == 1:
            continue
        else:

            line = mean_line(lines)
            # print('ok line', len(lines), line.length)

            ok_lines.append(line)
    return ok_lines

def probabilistic_hugh_transform(image, center, debug=False):
    image = image.squeeze()
    image = (image > 0).astype('bool') * 255

    edges = canny(image, 2, 1, 25)
    lines = probabilistic_hough_line(image, threshold=10, line_length=12,
                                     line_gap=3, seed=42)
    new_lines = []
    for start, end in lines:
        start_p = Point(*start)
        end_p = Point(*end)
        origin = Point(0,0)
        if end_p.distance(origin) < start_p.distance(origin):
            start_p, end_p = end_p, start_p
        new_lines.append(Line(start_p, end_p))
    lines = new_lines

    clustered_lines = cluster_lines(lines)
    if debug:
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        ax = axes.ravel()
        for l in clustered_lines:
            l.plot(ax=ax[0])
        ax[0].imshow(image, cmap=cm.gray)
        ax[0].set_title('Input image')
        center.plot(ax[0])

        ax[1].imshow(edges * 0)
        for line in lines:
            line.plot(ax[1])
        ax[1].set_xlim((0, image.shape[1]))
        ax[1].set_ylim((image.shape[0], 0))
        ax[1].set_title('Probabilistic Hough')
        center.plot(ax[1])

        for a in ax:
            a.set_axis_off()

        plt.tight_layout()
        plt.show()
    return clustered_lines

In [ ]:



out = cv2.VideoWriter(
    "watch-face-1.mov",
    cv2.VideoWriter_fourcc("M", "J", "P", "G"),
    10,
    (image_size[0], image_size[1]),
)
use_angle_model = False

for i, file in enumerate(tqdm(sorted(Path("./frames/").glob("*_0.jpg")))):
    with Image.open(file) as crop:
        crop_rotated_resized_np = ImageOps.pad(
            crop,
            image_size, BICUBIC
        )
        frame = np.array(crop_rotated_resized_np).copy()
        image_np = np.expand_dims(crop_rotated_resized_np, 0)
        predicted_hands = sm_model.predict(
            image_np
        )[0]
        predicted_hands = (predicted_hands > 0.1)
        predicted_hands = (predicted_hands * 255).astype('uint8').squeeze()


        predicted_kp = kp_model.predict(
            image_np
        )[0]

        scale_x = crop_rotated_resized_np.width / predicted.shape[0]
        scale_y = crop_rotated_resized_np.height / predicted.shape[1]

        outputs = convert_mask_outputs_to_keypoints(predicted_kp,
                                                    decode_hands_from_lines=True)


        center = outputs[0]
        hands_points =  extract_points_from_predictions(predicted_hands, center=center)

        crop_keypoints = [
         outputs[0],
            outputs[1],
            *hands_points,
        ]
        if len(hands_points) == 2:
            line_proposals = [
                Line(center, hands_points[0]),
                Line(center, hands_points[1]),

            ]
        else:
            line_proposals = []

        zeros = np.zeros_like(predicted_hands)
        predicted_all = np.stack((predicted_hands, zeros, zeros), axis=-1)
        predicted_frame = cv2.cvtColor(predicted_all, cv2.COLOR_RGB2BGR)
        for kp in crop_keypoints:
            predicted_frame = plot_kp_on_image(predicted_frame, kp)

        for line in line_proposals:
            predicted_frame = plot_line_on_image(predicted_frame, line)

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        overlay = cv2.addWeighted(frame, 0.45, predicted_frame, 0.55, 0)
        out.write(overlay)
out.release()
cv2.destroyAllWindows()

In [ ]:
plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
plt.show()